<a href="https://colab.research.google.com/github/sohambhattacharyya16/LLM_Pipelines/blob/main/Language_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 8.9 MB/

In [2]:
import os
import gradio as gr
import torch
from transformers import pipeline

In [3]:
#Load the ASR and Text Translation pipelines
asr_pipeline = pipeline("automatic-speech-recognition", model="distil-whisper/distil-small.en")
translator = pipeline(task="translation", model="facebook/nllb-200-distilled-600M")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/332M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [4]:
#Creating a function to transcribe speech. First we get the output as a text and then we feed that to the translator pipeline.

def transcribe_speech(filepath, target_language):
    if not filepath:
        return "No audio found, please retry.", []

    # Transcribe the speech
    output = asr_pipeline(filepath)
    transcription = output["text"]

    # Translate the transcribed text
    text_translated = translator(transcription, src_lang="eng_Latn", tgt_lang=target_language)

    return transcription, text_translated[0]['translation_text']

In [11]:
# Gradio interfaces for microphone and file upload
mic_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=[
        gr.Audio(sources="microphone", type="filepath"),
        gr.Dropdown(label="Target Language", choices=["ben_Beng", "hin_Deva", "fra_Latn", "spa_Latn", "deu_Latn"], value="ben_Beng")
    ],
    outputs=[
        gr.Textbox(label="Transcription", lines=3),
        gr.Textbox(label="Translation", lines=5)
    ],
    allow_flagging="never"
)

file_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=[
        gr.Audio(sources="upload", type="filepath"),
        gr.Dropdown(label="Target Language", choices=["ben_Beng", "hin_Deva", "fra_Latn", "spa_Latn", "deu_Latn"], value="ben_Beng")
    ],
    outputs=[
        gr.Textbox(label="Transcription", lines=3),
        gr.Textbox(label="Translation", lines=5)
    ],
    allow_flagging="never"
)

# Choices are presented as the model expects. The language codes are provded in "Languages in FLORES-200" in the link below
#https://github.com/facebookresearch/flores/blob/main/flores200/README.md#languages-in-flores-200

In [12]:
# Create the demo with tabbed interfaces
demo = gr.Blocks()

with demo:
    gr.TabbedInterface(
        [mic_transcribe, file_transcribe],
        ["Transcribe Microphone", "Transcribe Audio File"],
    )

# Launch the Gradio demo
if __name__ == "__main__":
    demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ed07bbcd0929b45be4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ed07bbcd0929b45be4.gradio.live


In [13]:
demo.close()

Closing server running on port: 7860


In [9]:
import gc
del asr_pipeline, translator